# Initial Setup

In [1]:
from keras import callbacks, models, metrics
from datahandler.constants import all_features, data_version, tensorboard_dir, location_labels, activity_labels
from datahandler.data_preprocessing import get_train_test_data, load_data_v3
from models.log_writer import LogWriter
from model.lstm import make_lstm_model_v1
from model.transformer import make_transformer_model_v1, make_transformer_model_v3
from model.cnn import make_cnn_model_v1, make_cnn_model_v2
import matplotlib.pyplot as plt
from utils import print_line_divider
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from datahandler.data_preprocessing_from_jupyter import load_all_raw_multitask_data
from audio.audio import play_training_is_complete
from sklearn.metrics import classification_report
import time

In [2]:
# DATA Loader
print_line_divider()
print("Preparing data...")
x_train, y_context_train, y_activity_train, x_test, y_context_test, y_activity_test = load_all_raw_multitask_data()
print("Train data shape: " + str(x_train.shape) + " | Train label shape: " + str(y_context_train.shape) + " and " + str(
    y_activity_train.shape))
print("Test data shape: " + str(x_test.shape) + " | Test label shape: " + str(y_context_test.shape) + " and " + str(
    y_activity_test.shape))
print_line_divider()


****************************************************
Preparing data...
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/mm5_datacollection.csv (1/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/pp1_datacollection.csv (2/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/ds2_datacollection.csv (3/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/ds1_datacollection.csv (4/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/tt4_datacollection.csv (5/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/os4_datacollection.csv (6/25)
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix_labeled/pp2_datacollection.csv (7/25)
Loading from file: /Users/

In [3]:
if len(x_train.shape) == 2:
    input_shape = (x_train.shape[1], 1)
else:
    input_shape = x_train.shape[1:]
print("Input shape: " + str(input_shape))

Input shape: (40, 9)


In [4]:
def check_distribution(labels_list, set_name, is_context):
    label_count_dict = {}
    for label_ind in labels_list:
        if label_ind in label_count_dict.keys():
            label_count_dict[label_ind] += 1
        else:
            label_count_dict[label_ind] = 1

    print_line_divider()
    print("Distribution for set " + set_name + "\n")
    for key in label_count_dict:
        key = int(key)
        if is_context:
            print(location_labels[key] + " count is " + str(label_count_dict[key]))
        else:
            print(activity_labels[key] + " count is " + str(label_count_dict[key]))


check_distribution(y_context_train, "Training set for context detection", True)
check_distribution(y_activity_train, "Training set for activity detection", False)
check_distribution(y_context_test, "Testing set for context detection", True)
check_distribution(y_activity_test, "Testing set for activity detection", False)

****************************************************
Distribution for set Training set for context detection

holdinginhand count is 1285
insidethepantpocket count is 1260
calling count is 1278
beingusedinhand count is 1294
insidethebag count is 1295
****************************************************
Distribution for set Training set for activity detection

standing count is 1974
walking count is 4438
****************************************************
Distribution for set Testing set for context detection

holdinginhand count is 215
insidethepantpocket count is 243
calling count is 223
beingusedinhand count is 207
insidethebag count is 205
****************************************************
Distribution for set Testing set for activity detection

standing count is 341
walking count is 752


# Main Training Configuration

In [5]:
# TRAINING CONFIGURATION
window_time_in_seconds = 2
window_size = 40
epochs = 100
batch_size = 32
validation_split = 15 / 85
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.005,
    name="Adam",
)
context_loss_function = "sparse_categorical_crossentropy"
activity_loss_function = "sparse_categorical_crossentropy"
callback_list = [
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
    callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)
]

# This is a section for multi-task learning

In [6]:
# EVALUATION
def test_model(context_ratio, activity_ratio, id):
    saved_model = "model/multitask" + str(id) + ".h5"
    _, model2 = make_transformer_model_v3(
        input_shape=input_shape,
        head_size=64,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=1,
        mlp_dropout=0.4,
        dropout=0.25,
    )
    model2.compile(
        optimizer=optimizer,
        loss={'context_output': context_loss_function, 'activity_output': activity_loss_function},
        loss_weights={'context_output': context_ratio, 'activity_output': activity_ratio},
        metrics=["sparse_categorical_accuracy"],
    )
    model2.fit(
        x_train,
        {'context_output': y_context_train, 'activity_output': y_activity_train},
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callback_list, # + [callbacks.ModelCheckpoint(saved_model, save_best_only=True, monitor="val_loss")],
        validation_split=validation_split,
        verbose=1,
        shuffle=True
    )
    # model2 = models.load_model(saved_model)
    _, _, _, context_acc, _ = model2.evaluate(x_test,
                                              {'context_output': y_context_test, 'activity_output': y_activity_test})
    print("Accuracy for context for ratio " + str(context_ratio) + " - " + str(activity_ratio) + " : ", context_acc)


In [7]:
test_model(5,1,1)

Epoch 1/100


2022-06-15 11:10:32.405914: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


165/165 [==============================] - 5s 24ms/step - loss: 6.0250 - context_output_loss: 1.0763 - activity_output_loss: 0.6434 - context_output_sparse_categorical_accuracy: 0.5242 - activity_output_sparse_categorical_accuracy: 0.6765 - val_loss: 5.9218 - val_context_output_loss: 1.0631 - val_activity_output_loss: 0.6063 - val_context_output_sparse_categorical_accuracy: 0.5133 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 0.0050
Epoch 2/100
165/165 [==============================] - 4s 24ms/step - loss: 4.2156 - context_output_loss: 0.7245 - activity_output_loss: 0.5932 - context_output_sparse_categorical_accuracy: 0.7142 - activity_output_sparse_categorical_accuracy: 0.6905 - val_loss: 5.0526 - val_context_output_loss: 0.9188 - val_activity_output_loss: 0.4585 - val_context_output_sparse_categorical_accuracy: 0.6829 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 0.0050
Epoch 3/100
165/165 [==============================] - 4s 26ms/step - loss: 3.

In [8]:
test_model(3,1,2)

Epoch 1/100
165/165 [==============================] - 10s 50ms/step - loss: 3.8999 - context_output_loss: 1.0865 - activity_output_loss: 0.6405 - context_output_sparse_categorical_accuracy: 0.5295 - activity_output_sparse_categorical_accuracy: 0.6756 - val_loss: 3.0274 - val_context_output_loss: 0.7999 - val_activity_output_loss: 0.6275 - val_context_output_sparse_categorical_accuracy: 0.5830 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 0.0012
Epoch 2/100
165/165 [==============================] - 8s 48ms/step - loss: 2.7597 - context_output_loss: 0.7147 - activity_output_loss: 0.6155 - context_output_sparse_categorical_accuracy: 0.7100 - activity_output_sparse_categorical_accuracy: 0.6920 - val_loss: 2.7868 - val_context_output_loss: 0.7304 - val_activity_output_loss: 0.5958 - val_context_output_sparse_categorical_accuracy: 0.6387 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 0.0012
Epoch 3/100
165/165 [==============================] - 9s 52ms/st

In [9]:
test_model(1.5,1,3)

Epoch 1/100
165/165 [==============================] - 14s 74ms/step - loss: 2.3278 - context_output_loss: 1.1240 - activity_output_loss: 0.6418 - context_output_sparse_categorical_accuracy: 0.4739 - activity_output_sparse_categorical_accuracy: 0.6794 - val_loss: 1.8886 - val_context_output_loss: 0.8439 - val_activity_output_loss: 0.6228 - val_context_output_sparse_categorical_accuracy: 0.5389 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 6.2500e-04
Epoch 2/100
165/165 [==============================] - 12s 74ms/step - loss: 1.8079 - context_output_loss: 0.7905 - activity_output_loss: 0.6222 - context_output_sparse_categorical_accuracy: 0.6714 - activity_output_sparse_categorical_accuracy: 0.6905 - val_loss: 1.9532 - val_context_output_loss: 0.8962 - val_activity_output_loss: 0.6088 - val_context_output_sparse_categorical_accuracy: 0.4885 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 6.2500e-04
Epoch 3/100
165/165 [==============================] - 1

In [10]:
test_model(1,1,4)

Epoch 1/100
165/165 [==============================] - 10s 52ms/step - loss: 2.0261 - context_output_loss: 1.3716 - activity_output_loss: 0.6545 - context_output_sparse_categorical_accuracy: 0.3629 - activity_output_sparse_categorical_accuracy: 0.6515 - val_loss: 1.7983 - val_context_output_loss: 1.1664 - val_activity_output_loss: 0.6319 - val_context_output_sparse_categorical_accuracy: 0.5345 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 1.0000e-04
Epoch 2/100
165/165 [==============================] - 8s 50ms/step - loss: 1.7474 - context_output_loss: 1.1126 - activity_output_loss: 0.6348 - context_output_sparse_categorical_accuracy: 0.5133 - activity_output_sparse_categorical_accuracy: 0.6777 - val_loss: 1.6918 - val_context_output_loss: 1.0660 - val_activity_output_loss: 0.6257 - val_context_output_sparse_categorical_accuracy: 0.5274 - val_activity_output_sparse_categorical_accuracy: 0.6811 - lr: 1.0000e-04
Epoch 3/100
165/165 [==============================] - 8s

# This section is for normal model training

In [13]:
_, normal_model = make_transformer_model_v1(
    input_shape=input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=1,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
# _, normal_model = make_cnn_model_v1(input_shape)
normal_model.summary()
normal_model.compile(
    optimizer=optimizer,
    loss=context_loss_function,
    metrics=["sparse_categorical_accuracy"],
)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 40, 9)]      0           []                               
                                                                                                  
 layer_normalization_10 (LayerN  (None, 40, 9)       18          ['input_6[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_5 (MultiH  (None, 40, 9)       9993        ['layer_normalization_10[0][0]', 
 eadAttention)                                                    'layer_normalization_10[0][0]'] 
                                                                                            

In [14]:
SAVED_BEST_MODEL_2 = "model/best_model_2.h5"
hist = normal_model.fit(
    x_train,
    y_context_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callback_list,
    validation_split=validation_split,
    verbose=1,
    shuffle=True
)
play_training_is_complete()

Epoch 1/100
165/165 [==============================] - 9s 51ms/step - loss: 1.4821 - sparse_categorical_accuracy: 0.3157 - val_loss: 1.1367 - val_sparse_categorical_accuracy: 0.5018 - lr: 1.0000e-04
Epoch 2/100
165/165 [==============================] - 9s 53ms/step - loss: 1.1991 - sparse_categorical_accuracy: 0.4481 - val_loss: 1.0538 - val_sparse_categorical_accuracy: 0.5318 - lr: 1.0000e-04
Epoch 3/100
165/165 [==============================] - 8s 51ms/step - loss: 1.1043 - sparse_categorical_accuracy: 0.5100 - val_loss: 0.9980 - val_sparse_categorical_accuracy: 0.5256 - lr: 1.0000e-04
Epoch 4/100
165/165 [==============================] - 9s 55ms/step - loss: 1.0479 - sparse_categorical_accuracy: 0.5511 - val_loss: 0.9820 - val_sparse_categorical_accuracy: 0.5141 - lr: 1.0000e-04
Epoch 5/100
165/165 [==============================] - 9s 54ms/step - loss: 1.0030 - sparse_categorical_accuracy: 0.5833 - val_loss: 0.9603 - val_sparse_categorical_accuracy: 0.5106 - lr: 1.0000e-04
Epoch

Input #0, wav, from '/var/folders/ph/8yv0q18n4mb1439tnf3rzcf40000gp/T/tmpmev8lt9f.wav':
  Duration: 00:00:02.27, bitrate: 705 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 1 channels, s16, 705 kb/s


In [15]:
# normal_model = models.load_model(SAVED_BEST_MODEL_2)
_, acc = normal_model.evaluate(x_test, y_context_test)
print("Accuracy for context: ", acc)

35/35 [==============================] - 1s 22ms/step - loss: 0.4115 - sparse_categorical_accuracy: 0.8472
Accuracy for context:  0.8472095131874084
